In [1]:
data_directory='C:\\Users\\tamyl\\OneDrive\\Desktop\\files\\MP_VERZEO'
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
##importing for model creation
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.utils.np_utils import to_categorical
import tensorflow 

In [2]:
IMG_SIZE=100
dataset=[]
categories=['with_mask','without_mask']#with_mask=0,without_mask=1
for category in categories:
    path=os.path.join(data_directory,category)
    class_num=categories.index(category)
    for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img))
        new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
        g_new_array=cv2.cvtColor(new_array,cv2.COLOR_BGR2GRAY)#gray_scale_array
        e_new_array=cv2.equalizeHist(g_new_array)#equalizeHist_array
        u_new_array=e_new_array/255#to ensure all values are between 0 ansd 1
        dataset.append([u_new_array,class_num])
        


In [3]:
random.shuffle(dataset)# for better training of the model(learning without confusion)
X=[]
y=[]
for features,labels in dataset:
    X.append(features)
    y.append(labels)
X=np.array(X)
y=np.array(y)

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [5]:
X_train,X_validation,y_train,y_validation=train_test_split(X_train,y_train,test_size=0.2)

In [6]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)
X_validation=X_validation.reshape(X_validation.shape[0],X_validation.shape[1],X_validation.shape[2],1)

In [7]:
a=X_test.shape
b=X_train.shape
c=X_validation.shape
print(a,b,c)

(276, 100, 100, 1) (880, 100, 100, 1) (220, 100, 100, 1)


In [8]:
y_train=to_categorical(y_train)#  onr-not enconding on the datasets
y_test=to_categorical(y_test)
y_validation=to_categorical(y_validation)

In [9]:
d=y_train.shape
e=y_test.shape
f=y_validation.shape
print(d,e,f)

(880, 2) (276, 2) (220, 2)


In [10]:
dataGen=ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.1,shear_range=0.1)
dataGen.fit(X_train)
batches=dataGen.flow(X_train,y_train,batch_size=32)#process 20 images at a time
X_batch,y_batch=next(batches)

In [13]:
model=Sequential()
model.add(Conv2D(70,(5,5),activation='relu',input_shape=(100,100,1)))
model.add(Conv2D(70,(5,5),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(70//2,(3,3),activation='relu'))
model.add(Conv2D(70//2,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(500,activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))#output layer
model.compile(Adam(lr=0.001),loss='binary_crossentropy',metrics='accuracy')  

In [14]:
final=model.fit(dataGen.flow(X_train,y_train,batch_size=32),validation_data=(X_validation,y_validation),epochs=25)

Epoch 1/25
28/28 [==============================] - 79s 3s/step - loss: 0.6971 - accuracy: 0.5270 - val_loss: 0.6401 - val_accuracy: 0.7045
Epoch 2/25
28/28 [==============================] - 78s 3s/step - loss: 0.6315 - accuracy: 0.6751 - val_loss: 0.5357 - val_accuracy: 0.7636
Epoch 3/25
28/28 [==============================] - 78s 3s/step - loss: 0.5648 - accuracy: 0.7285 - val_loss: 0.5805 - val_accuracy: 0.6818
Epoch 4/25
28/28 [==============================] - 78s 3s/step - loss: 0.5791 - accuracy: 0.7178 - val_loss: 0.4496 - val_accuracy: 0.7727
Epoch 5/25
28/28 [==============================] - 78s 3s/step - loss: 0.4799 - accuracy: 0.7733 - val_loss: 0.4365 - val_accuracy: 0.8182
Epoch 6/25
28/28 [==============================] - 76s 3s/step - loss: 0.4780 - accuracy: 0.7700 - val_loss: 0.3181 - val_accuracy: 0.8364
Epoch 7/25
28/28 [==============================] - 73s 3s/step - loss: 0.4032 - accuracy: 0.8133 - val_loss: 0.2384 - val_accuracy: 0.8818
Epoch 8/25
28/28 [==

In [ ]:
#final=model.fit(dataGen.flow(X_train,y_train,batch_size=40),validation_data=(X_validation,y_validation),epochs=25)

In [15]:
face_clsfr=cv2.CascadeClassifier('C:\\Users\\tamyl\\OneDrive\\Desktop\\haarcascade_frontalface_default.xml')#location

In [ ]:
#to be edited accordingly.
labels_dict={0:'with_mask',1:'without_mask'}
color_dict={0:(0,255,0),1:(0,0,255)}
size=4
webcam=cv2.VideoCapture(0)
#classifier=cv2.CasacadeClassifier('C:\\Users\\tamyl\\OneDrive\\Desktop\\haarcascade_frontalface_default.xml')
while True:
    (rval,im)=webcam.read()
    im=cv2.flip(im,1,1)
    mini=cv2.resize(im,(im.shape[1]// size,im.shape[0]//size))
    faces=face_clsfr.detectMultiScale(mini)
    for f in faces:
        (x,y,w,h)=[v*size for v in f]
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(-1,100,100,1))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,255,255),2)
        

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)

    if key == 27: 
        break

webcam.release()

cv2.destroyAllWindows()
        